In [1]:
# Imports
import os
import pandas as pd
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from django.apps import apps as django_apps
from edc_appointment.models import Appointment as CaregiverAppointment
from edc_base.utils import age, get_utcnow
from edc_constants.constants import FEMALE, MALE
from edc_visit_schedule.site_visit_schedules import site_visit_schedules
from flourish_child.helper_classes.utils import child_utils
from flourish_child.models import Appointment as ChildAppointment
from flourish_caregiver.helper_classes import SequentialCohortEnrollment
from pre_flourish.helper_classes import MatchHelper

match_helper_cls = MatchHelper()

In [2]:
def get_schedule_details(schedule_name):
    try:
        schedule = CohortSchedules.objects.get(schedule_name=schedule_name)
    except CohortSchedules.DoesNotExist:
        return None
    else:
        schedule_type = getattr(schedule, 'schedule_type', None)
        child_count = getattr(schedule, 'child_count', None)
        return schedule_type, child_count

In [3]:
def get_cohort_schedules(cohort_name, app_label, child_count=None, schedule_type=[]):
    schedules = CohortSchedules.objects.filter(cohort_name=cohort_name,
                                               onschedule_model__startswith=app_label)
    if child_count:
        schedules = schedules.filter(child_count=child_count)
    if schedule_type:
        schedules = schedules.filter(schedule_type__in=schedule_type)
    return schedules.values_list('schedule_name', flat=True)

In [4]:
def child_offstudy():
    identifiers = ChildOffStudy.objects.values_list('subject_identifier', flat=True)
    return identifiers

In [5]:
def check_fu_completed(subject_identifier, visit_model_cls):
    schedules = SubjectScheduleHistory.objects.filter(
        subject_identifier=subject_identifier,
        schedule_name__icontains='fu').values_list('schedule_name', flat=True)
    return visit_model_cls.objects.filter(
        subject_identifier=subject_identifier, schedule_name__in=schedules).exists()

In [6]:
def create_new_cohort(subject_identifier, cohort_name):
    Cohort.objects.update_or_create(subject_identifier=subject_identifier,
                                    name=cohort_name,
                                    defaults={'enrollment_cohort': False,})

In [7]:
def take_offschedule(schedule_obj):
    _, schedule = site_visit_schedules.get_by_onschedule_model_schedule_name(
        onschedule_model=schedule_obj.onschedule_model, name=schedule_obj.schedule_name)
    is_onschedule = schedule.is_onschedule(subject_identifier=schedule_obj.subject_identifier,
                                           report_datetime=schedule_obj.onschedule_datetime)
    if is_onschedule:
        schedule.take_off_schedule(subject_identifier=schedule_obj.subject_identifier,
                                   schedule_name=schedule_obj.schedule_name)

In [8]:
def enrol_onschedule(
    helper_cls, subject_identifier, base_appt_datetime, onschedule_datetime, new_schedule, is_caregiver=False):
    try:
        schedule = CohortSchedules.objects.get(schedule_name=new_schedule)
    except SubjectScheduleHistory.DoesNotExist:
        print(f'No schedule {new_schedule}')
    else:
        helper_cls.put_on_schedule(schedule.onschedule_model,
                                   schedule.schedule_name,
                                   subject_identifier,
                                   is_caregiver=is_caregiver,
                                   base_appt_datetime=base_appt_datetime,
                                   onschedule_datetime=onschedule_datetime)

In [9]:
def remove_fu_schedule(subject_identifier, schedule_names, appt_cls, visit_cls, child_idx=None):
    schedules = SubjectScheduleHistory.objects.filter(
        subject_identifier=subject_identifier, schedule_name__in=schedule_names)
    filter_childpid = Q()

    for schedule in schedules:
        onschedule_cls = django_apps.get_model(schedule.onschedule_model)
        if child_idx:
            filter_childpid = Q(child_subject_identifier=child_idx)
        try:
            model_obj = onschedule_cls.objects.get(filter_childpid,
                                                   subject_identifier=subject_identifier,
                                                   schedule_name=schedule.schedule_name)
        except onschedule_cls.DoesNotExist:
            print(f'No FU to remove, {subject_identifier}, {schedule.schedule_name}')
        else:
            appointments = appt_cls.objects.filter(subject_identifier=subject_identifier,
                                                   schedule_name=model_obj.schedule_name)
            visits = visit_cls.objects.filter(appointment__in=appointments)
            print(model_obj.schedule_name, visits, appointments)
            visits.delete()
            appointments.delete()
            model_obj.delete()
    schedules.delete()

In [10]:
def get_latest_schedule(subject_identifier, old_schedules):
    try:
        last_schedule = SubjectScheduleHistory.objects.filter(
            subject_identifier=subject_identifier, schedule_name__in=old_schedules).latest(
            'onschedule_datetime', 'created')
    except SubjectScheduleHistory.DoesNotExist:
        pass
    else:
        return last_schedule

In [11]:
def calc_child_bmi(subject_identifier):
    try:
        measurements = ChildClinicalMeasurements.objects.filter(
            child_visit__subject_identifier=subject_identifier).latest('report_datetime')
    except ChildClinicalMeasurements.DoesNotExist:
        return None
    else:
        return measurements.child_weight_kg / pow(measurements.child_height/100, 2)

In [13]:
def switch_to_sec(cohort, subject_identifier):
    """ Take participant both caregiver/child from primary to secondary aims cohort C
        @param cohort: current cohort instance
        @param subject_identifier: child's identifier
    """
    # Declares all necessary variables
    sq_enrolment_helper = SequentialCohortEnrollment(subject_identifier)
    caregiver_sidx = child_utils.caregiver_subject_identifier(subject_identifier)
    child_count = cohort.caregiver_child_consent.caregiver_visit_count

    # Remove child/caregiver from all FU schedules they may have been scheduled for
    child_fu_schedules = get_cohort_schedules(cohort.name, 'flourish_child', schedule_type=['sq_followup', 'followup'])
    remove_fu_schedule(subject_identifier, child_fu_schedules, ChildAppointment, ChildVisit)

    caregiver_fu_schedules = get_cohort_schedules(cohort.name, 'flourish_caregiver', child_count, ['sq_followup', 'followup'])
    remove_fu_schedule(caregiver_sidx, caregiver_fu_schedules, CaregiverAppointment, MaternalVisit, subject_identifier)

    # Get all related schedules for the current cohort, both child/caregiver
    child_schedules = get_cohort_schedules(cohort.name, 'flourish_child', )
    caregiver_schedules = get_cohort_schedules(cohort.name, 'flourish_caregiver', child_count)

    # Get child's latest onschedule instance and take them offschedule for it, if onschedule.
    child_latest_schedule = get_latest_schedule(subject_identifier, child_schedules)
    print('child last onschedule', child_latest_schedule.schedule_name)
    take_offschedule(schedule_obj=child_latest_schedule)

    # Same step above for caregiver
    caregiver_latest_schedule = get_latest_schedule(caregiver_sidx, caregiver_schedules)
    print('caregiver last onschedule', caregiver_latest_schedule.schedule_name)
    take_offschedule(schedule_obj=caregiver_latest_schedule)

    # Enrol both child/caregiver on secondary aims, at same timepoint they previously were on
    # Create new sec aims cohort instance
    create_new_cohort(subject_identifier, 'cohort_c_sec')

    # Get related sec aims schedule details for both child/caregiver and enrol onschedule
    schedule_type, _count = get_schedule_details(child_latest_schedule.schedule_name)

    try:
        enrol_visit = ChildVisit.objects.get(Q(schedule_name__icontains='enrol') |
                                             Q(schedule_name__icontains='sec') &
                                             ~Q(schedule_name__icontains='qt'),
                                            subject_identifier=subject_identifier,)
    except ChildVisit.DoesNotExist:
        pass
    else:
        child_new_schedule = get_cohort_schedules(
            'cohort_c_sec', 'flourish_child', schedule_type=[schedule_type])
        enrol_onschedule(sq_enrolment_helper,
                         subject_identifier,
                         enrol_visit.report_datetime,
                         child_latest_schedule.onschedule_datetime,
                         child_new_schedule[0])
        sq_enrolment_helper.delete_completed_appointments(ChildAppointment, subject_identifier,
                                                          child_new_schedule[0])

        schedule_type, _count = get_schedule_details(caregiver_latest_schedule.schedule_name)
        caregiver_new_schedule = get_cohort_schedules(
            'cohort_c_sec', 'flourish_caregiver', _count, [schedule_type])
        enrol_onschedule(sq_enrolment_helper,
                         caregiver_sidx,
                         enrol_visit.report_datetime,
                         caregiver_latest_schedule.onschedule_datetime,
                         caregiver_new_schedule[0],
                         is_caregiver=True)
        sq_enrolment_helper.delete_completed_appointments(CaregiverAppointment, caregiver_sidx,
                                                          caregiver_new_schedule[0])
        print(f'succesful switch {subject_identifier}')

In [14]:
# Remove participant's from the primary to the secondary aims, making sure they do not have any FUs done
# 2 participant's from group(s) BMI: <14.9 and Age: [9.5, 14) Gender: Female => Group1
# 3 participant's from group(s) BMI: 15-17.9 and Age: [9.5, 14) Gender: Female => Group2
# 5 participant's from group(s) BMI: 15-17.9 and Age: [9.5, 14) Gender: Male => Group3
# All offstudy participant's

offstudy = child_offstudy()
cohort_c = Cohort.objects.filter(name='cohort_c',
                                 current_cohort=True,
                                 exposure_status='EXPOSED')
offstudy_list = cohort_c.filter(subject_identifier__in=offstudy)

cohort_c_list = []

for cohort in cohort_c:
    subject_identifier = cohort.subject_identifier
    has_fu = check_fu_completed(subject_identifier, ChildVisit)
    if not has_fu:
        dob = cohort.caregiver_child_consent.child_dob
        gender = cohort.caregiver_child_consent.gender
        _age = age(dob, get_utcnow().date())
        _age = _age.years + _age.months/12 + _age.days/365
        _bmi = calc_child_bmi(subject_identifier)
        cohort_c_list.append(
            {'cohort_instance': cohort,
             'subject_identifier': subject_identifier,
             'gender': gender,
             'age': _age,
             'bmi': _bmi})


In [15]:
cohort_c_df = pd.DataFrame(cohort_c_list)

In [16]:
categories = [
    {'limit': 2, 'bmi_range': (0, 14.9), 'age_range': (9.5, 14), 'gender': FEMALE},
    {'limit': 3, 'bmi_range': (15, 17.9), 'age_range': (9.5, 14), 'gender': FEMALE},
    {'limit': 5, 'bmi_range': (15, 17.9), 'age_range': (9.5, 14), 'gender': MALE}]

participants = {f'group_{_count}': [] for _count, _ in enumerate(categories)}

# Filter the DataFrame and create lists of participants for each category
for _count, category in enumerate(categories):
    filtered_data = cohort_c_df[
        (cohort_c_df['age'] >= category['age_range'][0]) &
        (cohort_c_df['age'] < category['age_range'][1]) &
        (cohort_c_df['gender'] == category['gender']) &
        (cohort_c_df['bmi'] >= category['bmi_range'][0]) &
        (cohort_c_df['bmi'] <= category['bmi_range'][1])
    ][:category['limit']]

    participants[f'group_{_count}'] = filtered_data['cohort_instance'].tolist()

In [17]:
for _, cohorts in participants.items():
    for cohort in cohorts:
        print(_, cohort.subject_identifier)
        switch_to_sec(cohort, cohort.subject_identifier)

group_0 B142-040990117-9-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990117-9-10
group_0 B142-040990061-9-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990061-9-10
group_1 B142-040990156-7-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990156-7-10
group_1 B142-040990069-2-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990069-2-10
group_1 B142-040990022-1-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990022-1-10
group_2 B142-040990152-6-10
child last onschedule child_c_quart_schedule1
caregiver last onschedule c_quarterly1_schedule1
succesful switch B142-040990152-6-10
group_2 B142-040990033-8-10
child last o

In [19]:
cohorts = Cohort.objects.all()
for cohort in cohorts:
    cohort.save()